<a href="https://colab.research.google.com/github/dtabuena/Floxed_APOE/blob/main/Floxed_APOE_Run_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'Get Standard Modules'

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import os
from scipy.signal import butter,filtfilt
from tqdm import tqdm
import warnings
from IPython.display import clear_output
from datetime import datetime
import sys
import warnings
import shutil
from google.colab import files
warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)
clear_output(wait=False)



!pip install openpyxl
!pip install XlsxWriter


'''Get Repositories'''
try: shutil.rmtree('/content/EphysLib')
except: None

"run dtabuena's ephys notebooks"
!git clone https://github.com/dtabuena/EphysLib
to_import = [
            'ABF_Quality_Control.ipynb',
            'Basic_Ephys.ipynb',
            'Simple_ABF_tools.ipynb',             
            'Pipeline_Wrapper.ipynb'
            'fun_math.ipynb',
            'importing_abfs_from_dropbox.ipynb',
            'QC_recoding_dataframe.ipynb',
            'Analyzers/input_resistance_analyzer.ipynb',
            'Analyzers/gain_analyzer.ipynb',
            'Analyzers/latencey_analyzer.ipynb',
            'Analyzers/IV_analyzer.ipynb',
            'Analyzers/Vm_analyzer.ipynb',
            'Analyzers/membrane_analyzer.ipynb',
            'Analyzers/rheobase_analyzer.ipynb',
            'Pipeline_Wrapper.ipynb',
            ]
for i in to_import:
    f = '/content/EphysLib/' + i
    %run $f




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
##### Setup Protocol List
VC_prot = ['VC - MemTest-10ms-160ms',
           'VC - Multi IV - 150ms',]
IC_prot = ['IC - Gain - D20pA',
           'IC - Gain - D50pA',
           'IC - Rheobase',
           'IC - R input',
           'IC - Latentcy 800pA-1s',
           'VC - 3min GapFree',
           'I0 - 3min GapFree']

In [ ]:
dataset = {'data_name': 'Floxed_APOE',
           'data_source': "https://www.dropbox.com/sh/x4mmoozuo0dp1ed/AAB5Nz3_vSg3wqnfqwgrjNGKa?dl=0",
           'file_naming_scheme': ['Recording_date','GenoType','Sex','Age','Slice_Num','Cell_num','Cell_Type'],
           }

results = ephys_wrapper(dataset,VC_prot,IC_prot,strat_cols=['GenoType', 'Cell_Type'])

In [ ]:
strat_df_dict = results['strat_df_dict']

labels = ['Rec_date', 'Virus', 'GenoType', 'Sex', 'Age', 'Slice_Num', 'Cell_num', 'Cell_Type']

possible_cols = list(strat_df_dict[list(strat_df_dict.keys())[0]].columns)
alt_strat_groups = ['Cm_pc_10.0',
                    'Ra_160.0',
                    'Rm_160.0',
                    'Cmq_160.0',
                    'Gain_(Hz/pA)',
                    'Rheobase',
                    'AP_thresh_US',
                    'fast_after_hyperpol',
                    'Spike_latency_(ms)',
                    'Input_Resistance_MO',
                    'Rmp_mV',
                    [c for c in possible_cols if 'Stim_Levels_(pA)' in c],
                    [c for c in possible_cols if 'Spike_Counts' in c],
                    [c for c in possible_cols if 'IV_Early_(V_stim)' in c],
                    [c for c in possible_cols if 'IV_Steady_State_(I_mean)' in c],
                    [c for c in possible_cols if 'IV_Early_(I_peak)' in c],
                    ]

alt_strat_dict = restrat(strat_df_dict,alt_strat_groups  )
write_strat_dfs(alt_strat_dict, dataset['data_name']+'_results_stratified_alternate')